In [34]:
from pyspark.sql import SparkSession
from tabulate import tabulate
spark = SparkSession.builder.master("local[*]").getOrCreate()
expected = \
"""
  id
----
   0
   1
"""
df = spark.range(2)
got = "\n{}\n".format(tabulate(df.collect(), df.columns))
assert expected == got


['\n', ' ', ' ', 'i', 'd', '\n', '-', '-', '-', '-', '\n', ' ', ' ', ' ', '0', '\n', ' ', ' ', ' ', '1', '\n']
['\n', ' ', ' ', 'i', 'd', '\n', '-', '-', '-', '-', '\n', ' ', ' ', ' ', '0', '\n', ' ', ' ', ' ', '1', '\n']


In [38]:
from random_words import RandomWords
from pyspark.sql import Row
rows = [Row(i, RandomWords().random_word()) for i in range(100)]
df = spark.createDataFrame(rows, ["id", "word"])
df.show(4)

+---+----------+
| id|      word|
+---+----------+
|  0|  stocking|
|  1| gyroscope|
|  2| injectors|
|  3|letterhead|
+---+----------+
only showing top 4 rows



In [46]:
from pyspark.sql.functions import *
dateDf = df.withColumn("today", current_date())\
           .withColumn("now", current_timestamp())
dateDf.show(3, False)

+---+---------+----------+-----------------------+
|id |word     |today     |now                    |
+---+---------+----------+-----------------------+
|0  |stocking |2017-09-18|2017-09-18 21:28:17.711|
|1  |gyroscope|2017-09-18|2017-09-18 21:28:17.711|
|2  |injectors|2017-09-18|2017-09-18 21:28:17.711|
+---+---------+----------+-----------------------+
only showing top 3 rows



In [47]:
dateDf.printSchema()

root
 |-- id: long (nullable = true)
 |-- word: string (nullable = true)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [49]:
dateDf.select(date_add(col("today"), 5), date_sub(col("today"), 5)).show()

+------------------+------------------+
|date_add(today, 5)|date_sub(today, 5)|
+------------------+------------------+
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
|        2017-09-23|        2017-09-13|
+------------------+------------------+
only showing top 20 rows



In [74]:
today = col("today")
week_ago = date_sub(col("today"), 5)
exprs = [datediff(today, week_ago).alias("dif")]
exprs += [to_date(lit("2017-02-02")).alias("start")]
exprs += [to_date(lit("2020-10-02")).alias("end")]
exprs += [to_date(concat(lit("2020-"),lpad(col("id"), 2, "0"), lit("-01"))).alias("wrong")]
exprs += [unix_timestamp(lit("2019-02-02 02:02:02"), "yyyy-mm-dd HH:mm:ss").alias("unix")]
exprs += [from_unixtime(lit("1546383722"), "yyyy-mm-dd HH:mm:ss").alias("unix_time")]
dateDf2 = dateDf.where(col("id") % 2 == 0).select(exprs).withColumn("month_diff", months_between(col("start"), col("end")))
dateDf2.show(10)

+---+----------+----------+----------+----------+-------------------+----------+
|dif|     start|       end|     wrong|      unix|          unix_time|month_diff|
+---+----------+----------+----------+----------+-------------------+----------+
|  5|2017-02-02|2020-10-02|      null|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02|2020-02-01|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02|2020-04-01|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02|2020-06-01|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02|2020-08-01|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02|2020-10-01|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02|2020-12-01|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02|      null|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02|      null|1546390922|2019-02-02 00:02:02|     -44.0|
|  5|2017-02-02|2020-10-02| 

In [85]:
limit_rows = 3
print(tabulate(dateDf2.limit(limit_rows).collect(), dateDf2.columns))

  dif  start       end         wrong             unix  unix_time              month_diff
-----  ----------  ----------  ----------  ----------  -------------------  ------------
    5  2017-02-02  2020-10-02              1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-02-01  1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-04-01  1546390922  2019-02-02 00:02:02           -44


In [86]:
print(tabulate(dateDf2.na.drop("any").limit(limit_rows).collect(), dateDf2.columns))

  dif  start       end         wrong             unix  unix_time              month_diff
-----  ----------  ----------  ----------  ----------  -------------------  ------------
    5  2017-02-02  2020-10-02  2020-02-01  1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-04-01  1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-06-01  1546390922  2019-02-02 00:02:02           -44


In [87]:
print(tabulate(dateDf2.na.drop("all").limit(limit_rows).collect(), dateDf2.columns))

  dif  start       end         wrong             unix  unix_time              month_diff
-----  ----------  ----------  ----------  ----------  -------------------  ------------
    5  2017-02-02  2020-10-02              1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-02-01  1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-04-01  1546390922  2019-02-02 00:02:02           -44


In [88]:
print(tabulate(dateDf2.na.drop("any", subset=["wrong", "unix"]).limit(limit_rows).collect(), dateDf2.columns))

  dif  start       end         wrong             unix  unix_time              month_diff
-----  ----------  ----------  ----------  ----------  -------------------  ------------
    5  2017-02-02  2020-10-02  2020-02-01  1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-04-01  1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-06-01  1546390922  2019-02-02 00:02:02           -44


In [90]:
print(tabulate(dateDf2.na.drop("all", subset=["wrong", "unix"]).limit(limit_rows).collect(), dateDf2.columns))

  dif  start       end         wrong             unix  unix_time              month_diff
-----  ----------  ----------  ----------  ----------  -------------------  ------------
    5  2017-02-02  2020-10-02              1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-02-01  1546390922  2019-02-02 00:02:02           -44
    5  2017-02-02  2020-10-02  2020-04-01  1546390922  2019-02-02 00:02:02           -44


In [106]:
spark.range(5).selectExpr("stack(2, id)", "id").show()

+----+---+
|col0| id|
+----+---+
|   0|  0|
|null|  0|
|   1|  1|
|null|  1|
|   2|  2|
|null|  2|
|   3|  3|
|null|  3|
|   4|  4|
|null|  4|
+----+---+



In [109]:
spark.range(5).selectExpr("stack(2, id)").na.fill(34, subset=["col0"]).show()

+----+
|col0|
+----+
|   0|
|  34|
|   1|
|  34|
|   2|
|  34|
|   3|
|  34|
|   4|
|  34|
+----+



In [111]:
spark.range(5).selectExpr("stack(2, id)").na.replace([2], [44], "col0").show()

+----+
|col0|
+----+
|   0|
|null|
|   1|
|null|
|  44|
|null|
|   3|
|null|
|   4|
|null|
+----+



In [120]:
dateDf2.selectExpr("(unix, wrong) as complex", "start", "end").show(3)

+--------------------+----------+----------+
|             complex|     start|       end|
+--------------------+----------+----------+
|   [1546390922,null]|2017-02-02|2020-10-02|
|[1546390922,2020-...|2017-02-02|2020-10-02|
|[1546390922,2020-...|2017-02-02|2020-10-02|
+--------------------+----------+----------+
only showing top 3 rows



In [119]:
dateDf2.selectExpr("struct(unix, wrong) as complex", "start", "end")\
       .select("complex.unix").show(3)

+----------+
|      unix|
+----------+
|1546390922|
|1546390922|
|1546390922|
+----------+
only showing top 3 rows



In [124]:
dateDf2.select(struct(col("unix"), col("wrong")).alias("complex"))\
       .withColumn("expand", col("complex.wrong"))\
       .select("complex.*", "expand", "complex")\
       .show(3)

+----------+----------+----------+--------------------+
|      unix|     wrong|    expand|             complex|
+----------+----------+----------+--------------------+
|1546390922|      null|      null|   [1546390922,null]|
|1546390922|2020-02-01|2020-02-01|[1546390922,2020-...|
|1546390922|2020-04-01|2020-04-01|[1546390922,2020-...|
+----------+----------+----------+--------------------+
only showing top 3 rows



In [130]:
text_df = spark.read.text("*.py")

In [135]:
text_df.select(split(trim(col("value")), " ").alias("arr"))\
       .select(expr("arr[0]"), expr("arr").getItem(0))\
       .show()

+--------------------+--------------------+
|              arr[0]|              arr[0]|
+--------------------+--------------------+
|                 def|                 def|
|              import|              import|
|              import|              import|
|                    |                    |
|                   x|                   x|
|                   y|                   y|
|                    |                    |
|                with|                with|
|               reg_x|               reg_x|
|               reg_y|               reg_y|
|                    |                    |
|peachpy.x86_64.LO...|peachpy.x86_64.LO...|
|peachpy.x86_64.LO...|peachpy.x86_64.LO...|
|                 for|                 for|
|peachpy.x86_64.AD...|peachpy.x86_64.AD...|
|                    |                    |
|peachpy.x86_64.RE...|peachpy.x86_64.RE...|
|                    |                    |
|              py_fun|              py_fun|
|         print("done|         p

In [141]:
text_df.select(split(trim(col("value")), " ").alias("arr"))\
       .where(array_contains(col("arr"), "def"))\
       .withColumn("expld", explode(col("arr")))\
       .show(10, False)

+--------------------------------------+-------------------------------+
|arr                                   |expld                          |
+--------------------------------------+-------------------------------+
|[def, peachpy_one():]                 |def                            |
|[def, peachpy_one():]                 |peachpy_one():                 |
|[def, peachpy_conways_game_of_life():]|def                            |
|[def, peachpy_conways_game_of_life():]|peachpy_conways_game_of_life():|
|[def, test_this_test_will_pass(self):]|def                            |
|[def, test_this_test_will_pass(self):]|test_this_test_will_pass(self):|
|[def, test_not_equal(self):]          |def                            |
|[def, test_not_equal(self):]          |test_not_equal(self):          |
|[def, test_equality(self):]           |def                            |
|[def, test_equality(self):]           |test_equality(self):           |
+--------------------------------------+-----------

In [147]:
spark.range(5).select(collect_list(col("id"))).show()

+----------------+
|collect_list(id)|
+----------------+
| [0, 1, 2, 3, 4]|
+----------------+



In [148]:
dir(pyspark.sql.functions)

['AutoBatchedSerializer',
 'Column',
 'DataFrame',
 'DataType',
 'PickleSerializer',
 'SparkContext',
 'StringType',
 'UserDefinedFunction',
 '__all__',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '_binary_mathfunctions',
 '_create_binary_mathfunction',
 '_create_function',
 '_create_window_function',
 '_functions',
 '_functions_1_4',
 '_functions_1_6',
 '_functions_2_1',
 '_functions_2_2',
 '_parse_datatype_string',
 '_prepare_for_python_RDD',
 '_string_functions',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_window_functions',
 '_wrap_function',
 'abs',
 'acos',
 'add_months',
 'approxCountDistinct',
 'approx_count_distinct',
 'array',
 'array_contains',
 'asc',
 'ascii',
 'asin',
 'atan',
 'atan2',
 'avg',
 'base64',
 'bin',
 'bitwiseNOT',
 'blacklist',
 'broadcast',
 'bround',
 'cbrt',
 'ceil',
 'coalesce',
 'col',
 'collect_list',
 'collect_set',
 'column',
 'concat',
 'concat_ws',
 'conv',
 'corr',
 'cos',
 'cosh',
 'count',
 'countDistinct',
 'covar_p

In [159]:
from random_words import RandomWords
rows = [Row(i, RandomWords().random_word(), {"name": "nope", "name2": "again"}) for i in range(10)]
df_map = spark.createDataFrame(rows, ["id", "word", "d"])\
              .select(create_map(col("word"), col("id")).alias("map"), col("d"))\
              .select(explode(col("map")), col("d"))\
              .select(explode(col("d")), col("*"))\
              .show(10, False)

+-----+-----+------+-----+---------------------------------+
|key  |value|key   |value|d                                |
+-----+-----+------+-----+---------------------------------+
|name |nope |arrest|0    |Map(name -> nope, name2 -> again)|
|name2|again|arrest|0    |Map(name -> nope, name2 -> again)|
|name |nope |stuff |1    |Map(name -> nope, name2 -> again)|
|name2|again|stuff |1    |Map(name -> nope, name2 -> again)|
|name |nope |wrench|2    |Map(name -> nope, name2 -> again)|
|name2|again|wrench|2    |Map(name -> nope, name2 -> again)|
|name |nope |dam   |3    |Map(name -> nope, name2 -> again)|
|name2|again|dam   |3    |Map(name -> nope, name2 -> again)|
|name |nope |strobe|4    |Map(name -> nope, name2 -> again)|
|name2|again|strobe|4    |Map(name -> nope, name2 -> again)|
+-----+-----+------+-----+---------------------------------+
only showing top 10 rows



In [165]:
json = """
'{"myJSONKey": {"myJSONValue":[1,2,3]}}' as jsonstring
"""
json_df = spark.range(1).selectExpr(json)
print(tabulate(json_df.collect(), json_df.columns))
print("")
json_df.printSchema()

jsonstring
--------------------------------------
{"myJSONKey": {"myJSONValue":[1,2,3]}}

root
 |-- jsonstring: string (nullable = false)



In [170]:
json_df.select(get_json_object(col("jsonstring"), "$.myJSONKey.myJSONValue[0]"),
               json_tuple(col("jsonstring"), "myJSONKey")).show()

+-------------------------------------------------------+--------------------+
|get_json_object(jsonstring, $.myJSONKey.myJSONValue[0])|                  c0|
+-------------------------------------------------------+--------------------+
|                                                      1|{"myJSONValue":[1...|
+-------------------------------------------------------+--------------------+



In [174]:
rows = [Row(i, RandomWords().random_word()) for i in range(10)]
spark.createDataFrame(rows, ["id", "word"])\
     .selectExpr("(id, word) as complex")\
     .withColumn("json", to_json(col("complex")))\
     .show(10, False)

+--------------+----------------------------+
|complex       |json                        |
+--------------+----------------------------+
|[0,opinions]  |{"id":0,"word":"opinions"}  |
|[1,hilltops]  |{"id":1,"word":"hilltops"}  |
|[2,galleys]   |{"id":2,"word":"galleys"}   |
|[3,signaler]  |{"id":3,"word":"signaler"}  |
|[4,groan]     |{"id":4,"word":"groan"}     |
|[5,contention]|{"id":5,"word":"contention"}|
|[6,medicines] |{"id":6,"word":"medicines"} |
|[7,sight]     |{"id":7,"word":"sight"}     |
|[8,conspiracy]|{"id":8,"word":"conspiracy"}|
|[9,fists]     |{"id":9,"word":"fists"}     |
+--------------+----------------------------+

